In [11]:
from models.priors import *
x1 = jnp.array([1,2,3])
dist_euclid(x1,x1)

NameError: name 'jnp' is not defined

In [2]:
%run priors.py
x1 = jnp.array([1,2,3])
dist_euclid(x1,x1)

DeviceArray([[0., 1., 2.],
             [1., 0., 1.],
             [2., 1., 0.]], dtype=float32)

In [21]:
sys.path

['/Users/es/Dropbox/00 Oxford/96_cvae_jax_port/PriorCVAE_JAX',
 '/opt/anaconda3/envs/numpyro10_torch/lib/python38.zip',
 '/opt/anaconda3/envs/numpyro10_torch/lib/python3.8',
 '/opt/anaconda3/envs/numpyro10_torch/lib/python3.8/lib-dynload',
 '',
 '/opt/anaconda3/envs/numpyro10_torch/lib/python3.8/site-packages']